In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='06-sklearn.tree'
classifierName='01-DecisionTreeClassifier'
extraParameterName='1-Default'


print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Hyperparameters Results
# not too high to avoid overfitting (lower or equals to 12 ?)
max_depthFound=14
min_samples_leafFound=5
min_samples_splitFound=22
criterionFound='gini'

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound)


print('done')

In [ ]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrx.head(5)

# Test with default hyperparameters

issue overfitting (train good results, test poor results)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx0)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = DecisionTreeClassifier(random_state=42)
modelClf.fit(x_train, y_train)
print("# modelClf.get_depth()",modelClf.get_depth())

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result(packageName,classifierName,'1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')


fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
#fd.show_precision_recall_curve(modelClf,x_train,y_train,x_test,y_test)
fd.show_prediction_graph(modelClf, x_test,y_test)



# Hyperparameters tuning

In [ ]:
%%script false

from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(6,15),
    'min_samples_leaf': randint(5,25),
    'min_samples_split': randint(20,40),
}

res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#{'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 10, 'min_samples_split': 30}
#0.14124665444787682
#score   0.23259493670886075
#scoref1 0.23259493670886075

#{'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 25}
#0.1431167852901453
#score   0.23322683706070288
#scoref1 0.23322683706070288




In [ ]:
%%script false

modelClf = DecisionTreeClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': [12,13,14],
    'min_samples_leaf': [5,6,7],
    'min_samples_split': [22,23,24]
}

res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)


#{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 6, 'min_samples_split': 23}
#0.1708283458771155
#scoref1 0.24266455194290246

#{'criterion': 'gini', 'max_depth': 14, 'min_samples_leaf': 5, 'min_samples_split': 22}
#0.17773439471611638
#scoref1 0.26459143968871596



In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx0)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result(packageName,classifierName,'2-After tuning', duration_in_s)
f1Test,mccTest,rocTest=fd.print_scores(y_test, predsTest,'all')


f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
resultMd.update_performance_test_result(packageName,classifierName,'2-After tuning', f1Test,mccTest,rocTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD06A-DecisionTreeClassifierProbaHistogram' if saveImage else None)

diffF1=f1Train-f1Test
print("diffF1",diffF1)

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': modelClf.feature_importances_})
val=tmp[(tmp['Feature']=='trx_amount_log10')]['Feature importance']
amountImportance =val.values[0]
print("amount importance",amountImportance)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
# Plot the decision tree
plt.figure(figsize=(20, 10))
plot_tree(modelClf, filled=True, feature_names=dfTrx.columns, class_names=['Geniune', 'Fraudulent'])
plt.show()


In [ ]:
files = fd.getAllFiles()
predictors = fd.getPredictors(dfTrx0)

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx=fd.getStandardScaledData(file,source,predictors)
    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3],diffF1,amountImportance)

